In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mysql_connection as mc
%matplotlib inline

In [2]:
conn = mc.connect()

Please insert your password:········


# Sellers

Definition: *A user that has sold a product in a given time frame*

## 1. How many sellers do we have (only after Jan. 1st, 2017)?

In [4]:
sellers_query = """
SELECT DISTINCT
    oi.seller_id
FROM olist.order_items oi
    INNER JOIN olist.orders o
    ON oi.order_id = o.order_id
WHERE o.order_purchase_timestamp >= '2017-01-01'
"""

In [5]:
sellers = pd.read_sql(sellers_query, conn)

In [6]:
sellers.shape

(3068, 1)

We have had a total of 3068 unique sellers after Jan. 1st 2017

## 2. How much do they sell?

In [7]:
seller_qty_query = """
SELECT
    oi.seller_id
    ,COUNT(*)     AS items_sold
FROM olist.order_items oi
    INNER JOIN olist.orders o
    ON oi.order_id = o.order_id
WHERE o.order_purchase_timestamp >= '2017-01-01'
GROUP BY oi.seller_id
"""

seller_qty = pd.read_sql(seller_qty_query, conn)

In [9]:
seller_qty.shape

(3068, 2)

In [10]:
seller_qty.describe()

,items_sold
count,3068.000000
mean,36.597132
std,119.512140
min,1.000000
25%,2.000000
50%,8.000000
75%,25.000000
max,2033.000000
